In [1]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from experiments.Models import BertBaseline


2024-03-25 20:04:28.474110: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 20:04:28.474178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 20:04:28.554037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 20:04:28.721754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 20:04:29.629469: W tensorflow/compiler/tf2

In [3]:
model = BertBaseline()
model.load_model_weights('../models/bert_evidence_baseline_weights.h5')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
test_dataset = pd.read_csv('../data/emrqa_evidence_test.csv')
tokenized_test = tokenizer(test_dataset['question'].to_list(),
                           test_dataset['evidence'].to_list(),
                           padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
test_x = (tokenized_test['input_ids'], tokenized_test['attention_mask'])
test_y = np.stack((np.eye(512)[test_dataset['start_token']], np.eye(512)[test_dataset['end_token']]), axis=1)
results = model.model.predict(test_x, batch_size=32)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                      

In [4]:
starts = []
ends = []
for i in results:
    i = i.T
    starts.append(np.argmax(i[0]))
    ends.append(np.argmax(i[1]))
predictions = pd.DataFrame({"start": starts, "end": ends})
predictions

,start,end
0,25,27
1,40,40
2,20,23
3,33,38
4,10,13
...,...,...
6596,30,32
6597,15,16
6598,12,24
6599,22,27


In [5]:
starts_truth = []
ends_truth = []
for i in test_y:
    starts_truth.append(np.argmax(i[0]))
    ends_truth.append(np.argmax(i[1]))
truth = pd.DataFrame({"start": starts_truth, "end": ends_truth})
truth

,start,end
0,25,27
1,40,40
2,20,23
3,33,38
4,10,13
...,...,...
6596,30,32
6597,15,16
6598,12,24
6599,22,27


In [6]:
incorrect = 0
for ind, i in enumerate(truth):
    t = truth.iloc[ind].to_numpy()
    p = predictions.iloc[ind].to_numpy()
    if not (sum(t == p) == 2):
        incorrect += 1
print(incorrect)

0


In [6]:
sum(t == p)

2